In [74]:
import pandas as pd
import glob
import datetime
import time
import matplotlib.pyplot as plt
import pathlib
import re

In [17]:
path=str(pathlib.Path().absolute())

In [18]:
excelFile='/mnt/volume/jupyter/szokereso/vip_szotar_1.3.xlsx'

In [78]:
def doneTheseDays(client):

    pngs = glob.glob(client+'/szokeresores/dailyfreqs/*png')
    csvs = glob.glob(client+'/szokeresores/dailyfreqs/*csv')
    
    csvdates = [datetime.datetime(*[int(num) for num in re.findall(r'\d+', filename)]) for filename in csvs]
    pngdates = [datetime.datetime(*[int(num) for num in re.findall(r'\d+', filename)]) for filename in pngs]
    
    csvAndPngDates = list(set.intersection(set(csvdates),set(pngdates)))
    
    return csvAndPngDates

In [19]:
def generateFreqRes(filesThatDate,date,freqFilesPath):
        
    dfs=[]
    for file in filesThatDate:
        try:
            dfs.append(pd.read_csv(file))
        except pd.io.common.EmptyDataError:
            print('EmptyDataError: '+file)
            
    df = pd.concat([df[col] for df in dfs for col in df.columns if 'client' in col])
    df_value_counts = df.value_counts()
    df_value_counts.sort_values(ascending=True).to_csv(freqFilesPath+'dailyfreqfile_'+str(date).split(' ')[0]+'.csv')
    #plot = pd.read_csv(freqFilesPath+'dailyfreqfile_'+str(date).split(' ')[0]+'.csv').plot.barh()
    plot = pd.read_csv(freqFilesPath+'dailyfreqfile_'+str(date).split(' ')[0]+'.csv',index_col=0).plot.barh(legend=False)
    fig = plot.get_figure()
    fig.savefig(freqFilesPath+'barplot_'+str(date).split(' ')[0]+'.png', bbox_inches='tight')
    plt.close()

In [20]:
def get_files_sorted_by_date_after_a_date(look_for_this_pattern,cutoffdate):
    
    csvs = glob.glob(look_for_this_pattern)
    datetimes=[datetime.datetime(int(each.split('/')[-1][37:41]),
                                 int(each.split('/')[-1][42:44]),
                                 int(each.split('/')[-1][45:47]),
                                 int(each.split('/')[-1][48:50]),
                                 int(each.split('/')[-1][51:53])) for each in csvs]

    dt_csvs_filtered=[[dt, csv] for dt, csv in zip(datetimes,csvs) if dt >= datetime.datetime(*cutoffdate)]
    
    sorted_filtered_csvs = [csv
                        for _, csv in sorted(
                                         zip([eachpair[0] for eachpair in dt_csvs_filtered],
                                             [eachpair[1] for eachpair in dt_csvs_filtered]))]
    
    return sorted_filtered_csvs

In [21]:
def getFilesOnGivenDate(day,momFilesPathAndWildCard):
    
    dayafterday = day + datetime.timedelta(days=1)
    
    return list(
    set(get_files_sorted_by_date_after_a_date(momFilesPathAndWildCard,[day.year,day.month,day.day,0]))-\
    set(get_files_sorted_by_date_after_a_date(momFilesPathAndWildCard,[dayafterday.year,dayafterday.month,dayafterday.day,0])))

In [97]:
def doThisWhenNotEnoughFilesForDay(sleepThisMuch=60*60,day=None):
    #print('Not enough files found. '+str(day)+' Sleeping '+str(sleepThisMuch/60)+' minutes.')
    print('Looped through all clients. Sleeping ' + str(sleepThisMuch / 60) + 'minutes')
    time.sleep(sleepThisMuch)

In [23]:
def didMoreThanThisManySecondsElapsedSinceEndOfThatDay(singleDate,seconds):
    return (datetime.datetime.now() - (singleDate + datetime.timedelta(days=1))).total_seconds() > seconds

In [24]:
def getClients(excelFile=excelFile):
        
    xl = pd.ExcelFile(excelFile)
    sheetNames = [sheetName.strip() for sheetName in xl.sheet_names]
        
    return sheetNames

'/mnt/volume/jupyter/szokereso/negyedikfeladatUjra'

In [114]:
clients = getClients()

clients = ['MSZP']

while True:
    for client in clients:

        freqFilesPath = path+'/'+client+'/szokeresores/dailyfreqs/'
        momFilesPathAndWildCard  = path+'/'+client+'/szokeresores/clientreszurt/live_updated3_dict_only_1client_data_*_dfUnifiedNanFilteredOnlySomeColsUpdated3.csv'

        start_date = datetime.datetime(2020,7,1)
        day_count = 10000

        doneThese = doneTheseDays(client)
        for single_date in (start_date + datetime.timedelta(n) for n in range(day_count)):
            if single_date not in doneThese:
                filesThatDate = getFilesOnGivenDate(single_date,momFilesPathAndWildCard)
                if len(filesThatDate) < 12 and not didMoreThanThisManySecondsElapsedSinceEndOfThatDay(single_date, 600):
                    #doThisWhenNotEnoughFilesForDay(sleepThisMuch=10*60,day=single_date)
                    break
                else:
                    generateFreqRes(filesThatDate,single_date,freqFilesPath)
                    print(client)
                    print(single_date)

    doThisWhenNotEnoughFilesForDay(sleepThisMuch=10*60,day=single_date)


Looped through all clients. Sleeping 10.0minutes


KeyboardInterrupt: 

In [116]:
date=single_date
dfs=[]
for file in filesThatDate:
    try:
        dfs.append(pd.read_csv(file))
    except pd.io.common.EmptyDataError:
        print('EmptyDataError: '+file)

df = pd.concat([df[col] for df in dfs for col in df.columns if 'client' in col])
df_value_counts = df.value_counts()
df_value_counts.sort_values(ascending=True).to_csv(freqFilesPath+'dailyfreqfile_'+str(date).split(' ')[0]+'.csv')
#plot = pd.read_csv(freqFilesPath+'dailyfreqfile_'+str(date).split(' ')[0]+'.csv').plot.barh()
plot = pd.read_csv(freqFilesPath+'dailyfreqfile_'+str(date).split(' ')[0]+'.csv',index_col=0).plot.barh(legend=False)
fig = plot.get_figure()
fig.savefig(freqFilesPath+'barplot_'+str(date).split(' ')[0]+'.png', bbox_inches='tight')
plt.close()

In [117]:
dfs[0]

,Unnamed: 0,DOC_ID,TITLE,SCRAPETIME,TEXT,OUTLET,clientdict0clssd,clientdict1clssd,clientdict2clssd,clientdict3clssd,clientdict4clssd,clientdict5clssd,clientdict6clssd,clientdict7clssd,clientdict8clssd,clientdict9clssd
0,0,1,Elhunyt Sváby Lajos festőművész,2020-08-14 04:00:13,\n\n __\n\n__ __ __\n\n__\n\n * Műsorok __\n ...,hang.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,Vallási szekták miatt harapódzik el újra a kor...,2020-08-14 04:00:13,\n\n __\n\n__ __ __\n\n__\n\n * Műsorok __\n ...,hang.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,Vidnyánszky: Nagyon komoly nemzetközi háttérha...,2020-08-14 04:00:13,\n\n__\n\n__ __ __\n\n__\n\n * Műsorok __\n ...,hang.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,Kamuvideót osztott meg a közmédia azt állítva ...,2020-08-14 04:00:13,\n\n __\n\n__ __ __\n\n__\n\n * Műsorok __\n ...,hang.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,Szeptemberben együtt avat a két Orbán,2020-08-14 04:00:13,\n\n__\n\n__ __ __\n\n__\n\n * Műsorok __\n ...,hang.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3555,3555,3556,Különleges programot rendeznek a szentendrei S...,2020-08-14 04:00:15,\n\nAz augusztus 20-i hosszú hétvége a Határta...,www.origo.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3556,3556,3557,Háromgólos vereséggel kezdte a rájátszást az N...,2020-08-14 04:00:15,\n\nA Vancouver Canucks szerdán 5-2-re megnyer...,www.origo.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3557,3557,3558,A gyerekeiket is magukkal vitték a kocsmába az...,2020-08-14 04:00:15,"\n\nEgy pécsi pár rendszeresen ivott, és ebben...",www.origo.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3558,3558,3559,Rétvári Bence: Tíz év alatt hét helyet javítot...,2020-08-14 04:00:15,\n\nAz Eurostat felmérése szerint tíz év alatt...,www.origo.hu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
